In [1]:
import re
import joblib
import numpy as np
import pandas as pd
from tqdm import tqdm

In [2]:
import plotly.express as px
import plotly.graph_objects as go

### Importing dataframes

In [3]:
from gentrification import load_dataframes

dataframes = load_dataframes()

In [4]:
age = dataframes['final_age']
rent = dataframes['final_rent']
area = dataframes['final_area']
migr = dataframes['final_migr']

### Data exploration

In [5]:
for i in age.LOR.unique():
    if i not in migr.LOR.unique():
        print(i)

121030
123020


In [6]:
age[age.LOR=='121030']

,Bezirk,Bezirksregion,LOR,Bezirksname,year,total,below_6,6-15,15-18,18-27,27-45,45-55,55-65,65_above
128,12,1030,121030,NaN,2007,48661,2371,3533,1295,4155,13964,7064,5807,10472
263,12,1030,121030,NaN,2008,48892,2482,3457,1247,4244,13826,7283,5773,10580


In [7]:
age[age.LOR=='123020']

,Bezirk,Bezirksregion,LOR,Bezirksname,year,total,below_6,6-15,15-18,18-27,27-45,45-55,55-65,65_above
133,12,3020,123020,NaN,2007,43909,2687,4549,1670,3534,10175,5735,6000,9559
268,12,3020,123020,NaN,2008,43109,2711,4572,1558,3502,9759,5673,5637,9697


In [8]:
# excluding years 2007 and 2008 for age and migr due to incomplete and inconsistent data

print(age[age['year']>=2009].shape)
print(migr[migr['year']>=2009].shape)

(1529, 14)
(1529, 30)


In [9]:
age = age[age['year']>=2009].copy()
#migr = migr[migr['year']>=2015].copy()
migr = migr[migr['year']>=2009].copy()

### Merging dataframes

In [10]:
df = age.merge(migr, on=['LOR', 'year', 'Bezirk'], how='outer')\
        .merge(rent, on=['LOR', 'year'])\
        .merge(area, on='LOR')

In [11]:
for col in ['lon', 'lat']:
    df.drop(col, axis=1, inplace=True)

### Distributing foreigner numbers according to population of LOR in comparison to Bezirk for years 2009-2014

In [12]:
df['total_Bezirk'] = df.groupby(['Bezirk', 'year'])['total'].transform(sum)

In [13]:
df['perc_Bezirk_total'] = df['total'] / df['total_Bezirk']

In [14]:
cols = ['total_foreigners', 'EU', 'FRA', 'GRC', 'ITA', 'AUT', 'ESP',
        'UK', 'POL', 'BGR', 'ROU', 'HRV', 'former_JUGO', 'BIH', 'SRB',
        'former_UDSSR', 'RUS', 'UKR', 'KAZ', 'Islamic_countries', 'TUR', 'IRN',
        'LBN', 'SYR', 'VNM', 'USA', 'not_identified']

mask_years = df['year']<=2014

for col in cols:
    df.groupby(['Bezirk', 'year'])[col].transform(sum)
    df.loc[mask_years, '_'.join([col, 'new'])] = df['perc_Bezirk_total'] * df[col]
    df.loc[~mask_years, '_'.join([col, 'new'])] = df[col]
    df.drop(col, axis=1, inplace=True)

In [15]:
cols_new = ['_'.join([c, 'new']) for c in cols]

df.columns = ['Bezirk', 'Bezirksregion', 'LOR', 'Bezirksname', 'year', 'total',
            'below_6', '6-15', '15-18', '18-27', '27-45', '45-55', '55-65',
            '65_above', 'PLZ', 'apt_size', 'sqm_price_all', 'rent_cold',
            'rent_warm', 'empty', 'net_income', 'rent_cold_fraction', 'rent_warm_fraction',
            'sqm_price_lower', 'sqm_price_upper', 'no_offers', 'area', 'area_sqkm',
            'living_area', 'leisure_area', 'leisure_part', 'lower_quality',
            'average_quality', 'upper_quality', 'urban_style', 'total_Bezirk',
            'perc_Bezirk_total'] + cols

### Fill cells with NaN values

In [16]:
df.loc[:, df.isna().any()].columns

Index(['empty', 'no_offers', 'urban_style', 'FRA', 'GRC', 'AUT', 'ESP', 'UK',
       'BGR', 'ROU', 'HRV', 'BIH', 'SRB', 'Islamic_countries', 'IRN', 'LBN',
       'SYR', 'not_identified'],
      dtype='object')

In [17]:
#fill nan values (SYR): fill with 0
df['SYR'].fillna(0, inplace=True)


#fill nan values (all but SYR): forward fill with value of following year
fill_cols = ['FRA', 'GRC', 'AUT', 'ESP', 'UK', 'BGR', 'ROU','HRV',
             'BIH', 'SRB', 'Islamic_countries', 'IRN', 'LBN', 'not_identified',]

for col in cols:
    for l in df['LOR'].unique():
        df.loc[df['LOR']==l, col] = df.loc[df['LOR']==l, col].fillna(method='backfill')

In [18]:
#drop columns empty  no_offers due to to many missing years or irrelevance
df = df.drop(['empty', 'no_offers'], axis=1)

In [19]:
df.loc[:, df.isna().any()].columns

Index(['urban_style'], dtype='object')

### Create columns for remaining population

In [20]:
df['DEU'] = df['total'] - df['total_foreigners']
df['EU_other'] = df['EU'] - df[['FRA','GRC', 'ITA', 'AUT', 'ESP', 'UK', 'POL', 'BGR', 'ROU', 'HRV']].sum(axis=1)
df['JUGO_other'] = df['former_JUGO'] - df[['BIH', 'SRB']].sum(axis=1)
df['UDSSR_other'] = df['former_UDSSR'] - df[['RUS', 'UKR', 'KAZ']].sum(axis=1)
df['Islamic_other'] = df['Islamic_countries'] - df[['TUR', 'IRN', 'LBN', 'SYR']].sum(axis=1)
df['other'] = df['total_foreigners'] - df[['EU', 'former_JUGO', 'former_UDSSR', 'Islamic_countries', 'VNM', 'USA', 'not_identified']].sum(axis=1)

### Create age categories

In [21]:
df['boomers'] = df['55-65'] + df['65_above']
df['gen_x'] = df['45-55']
df['millenials'] = df['18-27'] + df['27-45']
df['children'] = df['below_6'] + df['6-15'] + df['15-18']

### Fix Bezirksnamen

In [22]:
df.loc[df['Bezirk']=='12', 'Bezirksname'].unique()

array(['Ost 1 - Reginhardstraße', 'Ost 1', 'Ost 2 - Alt-Reinickendorf',
       'Ost 2', 'West 1 - Tegel-Süd/Flughafensee', 'West 1',
       'West 4 - Auguste-Viktoria-Allee', 'West 4',
       'West 5 - Tegel/Tegeler Forst', 'West 5',
       'West 2 - Heiligensee/Konradshöhe', 'West 2',
       'Nord 1 - Frohnau/Hermsdorf', 'Nord 1',
       'Nord 2 - Waidmannslust/Wittenau/Lübars', 'Nord 2',
       'MV 1 - Märkisches Viertel', 'MV 1 (Märkisches Viertel 1)',
       'MV 2 - Rollbergesiedlung', 'MV 2 (Märkisches Viertel 2)',
       'West 3 - Borsigwalde/Freie Scholle', 'West 3'], dtype=object)

In [23]:
name = {'Ost 1': 'Ost 1 - Reginhardstraße',
       'Ost 2': 'Ost 2 - Alt-Reinickendorf',
       'West 1': 'West 1 - Tegel-Süd/Flughafensee',
       'West 4': 'West 4 - Auguste-Viktoria-Allee', 
       'West 5': 'West 5 - Tegel/Tegeler Forst',
       'West 2': 'West 2 - Heiligensee/Konradshöhe', 
       'Nord 1': 'Nord 1 - Frohnau/Hermsdorf', 
       'Nord 2': 'Nord 2 - Waidmannslust/Wittenau/Lübars', 
       'MV 1 (Märkisches Viertel 1)': 'MV 1 - Märkisches Viertel', 
       'MV 2 (Märkisches Viertel 2)': 'MV 2 - Rollbergesiedlung', 
       'West 3': 'West 3 - Borsigwalde/Freie Scholle'}

df.replace(name, inplace=True)

### Fix Name

In [24]:
names = {'00': 'Berlin',
        '01': 'Mitte',
        '02': 'Friedrichshain-Kreuzberg',
        '03': 'Pankow',
        '04': 'Charlottenburg-Wilmersdorf',
        '05': 'Spandau',
        '06': 'Steglitz-Zehlendorf',
        '07': 'Tempelhof-Schöneberg',
        '08': 'Neukölln',
        '09': 'Treptow-Köpenick',
        '10': 'Marzahn-Hellersdorf',
        '11': 'Lichtenberg',
        '12': 'Reinickendorf'}

df['Name'] = df['Bezirk'].map(names)

### Fix outliers

LORs Allende-Viertel, Köpenick Süd and Forst Grunewald show some unusual developments or values.

In [25]:
fig = px.line(df[df['Bezirksname'].isin(['Allende-Viertel', 'Köpenick Süd', 'Forst Grunewald', 'Grunewald'])],
              x='year',
              y='total',
              hover_name='Bezirksname',
              color='Bezirksname',
              range_y=(0,24000),
              width=900,
              height=500)
#fig.show()

Effective 1.1.2019 specific areas were switched between Köpenick Süd and Allende-Viertel. Forst Grunewald has less than 75 inhabitants.

Solution:

Combine Köpenick Süd and Allende-Viertel to LOR Allende-Viertel/Köpenick Süd.

Combine Forst Grunewald with Grunewald.

In [26]:
cols_sum = ['total', 'below_6', '6-15', '15-18', '18-27', '27-45', '45-55', '55-65',
           '65_above', 'area', 'living_area', 'area_sqkm',
           'leisure_area', 
           'total_foreigners',
           'EU', 'FRA', 'GRC', 'ITA', 'AUT', 'ESP', 'UK', 'POL', 'BGR', 'ROU',
           'HRV', 'former_JUGO', 'BIH', 'SRB', 'former_UDSSR', 'RUS', 'UKR', 'KAZ',
           'Islamic_countries', 'TUR', 'IRN', 'LBN', 'SYR',
           'VNM', 'USA', 'not_identified']

cols_mean = ['apt_size', 'sqm_price_all', 'rent_cold',
             'rent_warm', 'net_income', 'rent_cold_fraction', 'rent_warm_fraction',
             'sqm_price_lower', 'sqm_price_upper',
             'lower_quality', 'average_quality', 'upper_quality',
             'perc_Bezirk_total']
        
for year in df['year'].unique():
    for c1 in cols_sum:
        df.loc[(df['Bezirksname']=='Allende-Viertel') & (df['year']==year), c1] = \
            float(df.loc[(df['Bezirksname']=='Allende-Viertel') & (df['year']==year), c1]) \
            + float(df.loc[(df['Bezirksname']=='Köpenick Süd') & (df['year']==year), c1])
        
        df.loc[(df['Bezirksname']=='Grunewald') & (df['year']==year), c1] = \
            float(df.loc[(df['Bezirksname']=='Grunewald') & (df['year']==year), c1]) \
            + float(df.loc[(df['Bezirksname']=='Forst Grunewald') & (df['year']==year), c1])
        
    for c2 in cols_mean:
        df.loc[(df['Bezirksname']=='Allende-Viertel') & (df['year']==year), c2] = \
            (float(df.loc[(df['Bezirksname']=='Allende-Viertel') & (df['year']==year), c2]) \
            + float(df.loc[(df['Bezirksname']=='Köpenick Süd') & (df['year']==year), c2])) / 2

                        
df.drop(df[df['Bezirksname']=='Köpenick Süd'].index, axis=0, inplace=True)
df.drop(df[df['Bezirksname']=='Forst Grunewald'].index, axis=0, inplace=True)
                        
df.replace({'Allende-Viertel': 'Allende-Viertel/Köpenick Süd'}, inplace=True)

In [27]:
fig = px.line(df[df['Bezirksname'].isin(['Allende-Viertel/Köpenick Süd', 'Grunewald'])],
              x='year',
              y='total',
              hover_name='Bezirksname',
              color='Bezirksname',
              range_y=(0, 24000),
              width=900,
              height=500)
#fig.show()

### Percentages

In [28]:
def percentage(df, total_col, col_name):
    df['_'.join([col_name, 'perc'])] = df[col_name] / df[total_col]
    #df.drop(col_name, axis=1, inplace=True)
    return df

In [29]:
cols = ['below_6', '6-15', '15-18', '18-27', '27-45', '45-55', '55-65',
       '65_above', 'boomers', 'gen_x', 'millenials', 'children']

df_perc = df.copy()

for col in cols:
    percentage(df_perc, 'total', col)

In [30]:
cols = ['EU', 'FRA', 'GRC', 'ITA', 'AUT', 'ESP',
        'UK', 'POL', 'BGR', 'ROU', 'HRV', 'former_JUGO', 'BIH', 'SRB',
        'former_UDSSR', 'RUS', 'UKR', 'KAZ', 'Islamic_countries', 'TUR', 'IRN',
        'LBN', 'SYR', 'VNM', 'USA', 'not_identified', 'DEU', 'EU_other',
        'JUGO_other', 'UDSSR_other', 'Islamic_other', 'other']

for col in cols:
    percentage(df_perc, 'total_foreigners', col)

In [31]:
df_perc['total_foreigners_perc'] = df_perc['total_foreigners'] / df_perc['total']

In [32]:
df_perc['pop_per_sqm'] = df_perc['total'] / df_perc['area_sqkm']

In [33]:
df_perc.shape

(1507, 119)

In [34]:
joblib.dump(df_perc, './dataframes/final_all.pkl')

['./dataframes/final_all.pkl']

### Create mapper dictionaries

In [35]:
lor_mapper = dict(zip(df_perc.LOR.unique(), df_perc.Bezirksname.unique()))

In [36]:
region_mapper = {
    'FRA': 'EU',
    'GRC': 'EU',
    'ITA': 'EU',
    'AUT': 'EU',
    'ESP': 'EU',
    'UK': 'EU',
    'POL': 'EU',
    'BGR': 'EU',
    'ROU': 'EU',
    'HRV': 'EU',
    'EU_other': 'EU',
    'DEU': 'EU',
    'BIH': 'former_JUGO',
    'SRB': 'former_JUGO',
    'JUGO_other': 'former_JUGO',
    'RUS': 'former_UDSSR',
    'UKR': 'former_UDSSR',
    'KAZ': 'former_UDSSR',
    'UDSSR_other': 'former_UDSSR',
    'TUR': 'Islamic_countries',
    'IRN': 'Islamic_countries',
    'LBN': 'Islamic_countries',
    'SYR': 'Islamic_countries',
    'Islamic_other': 'Islamic_countries',
    'VNM': 'Other',
    'USA': 'Other',
    'not_identified': 'Other',
    'other': 'Other'
}

In [37]:
migr_mapper = {
    'total': 0,
    'migr': 1,
    'no_migr': 2,
    'EU': 3,
    'Islamic_countries': 4,
    'Other': 5,
    'former_JUGO': 6,
    'former_UDSSR': 7,
    'AUT': 8,
    'BGR': 9,
    'DEU': 10,
    'ESP': 11,
    'EU_other': 12,
    'FRA': 13,
    'GRC': 14,
    'HRV': 15,
    'ITA': 16,
    'POL': 17,
    'ROU': 18,
    'UK': 19,
    'IRN': 20,
    'Islamic_other': 21,
    'LBN': 22, 
    'SYR': 23,
    'TUR': 24,
    'USA': 25,
    'VNM': 26, 
    'not_identified': 27, 
    'other': 28, 
    'BIH': 29,
    'JUGO_other': 30,
    'SRB': 31, 
    'KAZ': 32, 
    'RUS': 33, 
    'UDSSR_other': 34,
    'UKR': 35
}

In [38]:
age_mapper = {
    'below_6': 'Children',
    '6-15': 'Children',
    '15-18': 'Children',
    '18-27': 'Millenials',
    '27-45': 'Millenials',
    '45-55': 'Generation X',
    '55-65': 'Baby<br>Boomers',
    '65_above': 'Baby<br>Boomers',
    'below_6_perc': 'Children (%)',
    '6-15_perc': 'Children (%)',
    '15-18_perc': 'Children (%)',
    '18-27_perc': 'Millenials (%)',
    '27-45_perc': 'Millenials (%)',
    '45-55_perc': 'Generation X (%)',
    '55-65_perc': 'Baby<br>Boomers (%)',
    '65_above_perc': 'Baby<br>Boomers (%)'
}

In [39]:
features_long = {
    'sqm_price_all': 'Square Meter Price (in EUR)',
    'total': 'Total Population',
    'pop_per_sqm': 'Population per km2',
    'area_sqkm': 'Area (in km2)',
    'sqm_price_lower': 'Square Meter Price (in EUR) - lower',
    'sqm_price_upper': 'Square Meter Price (in EUR) - upper',
    'rent_cold': 'Rent cold (in EUR)',
    'apt_size': 'Apartment Size (in m2)',
    'net_income': 'Household Net Income (in EUR)',
    'upper_quality': 'Percentage Upper Quality Housing',
    'total_foreigners_perc': 'Percentage Pop. with Migration History',
    'leisure_part': 'Percentage Park Area',
    'cluster': 'Cluster',
    'factor': 'GentriFactor',
    
    'below_6': '<6',
    '6-15': 'Age Bracket 6-15',
    '15-18': 'Age Bracket 15-18',
    '18-27': 'Age Bracket 18-27',
    '27-45': 'Age Bracket 27-45',
    '45-55': 'Age Bracket 45-55',
    '55-65': 'Age Bracket 55-65',
    '65_above': '65+',
    'millenials': 'Millenials',
    'boomers': 'Baby<br>Boomers',
    'gen_x': 'Generation X',
    'children': 'Children',
    'below_6_perc': 'Percentage Population < 6',
    '6-15_perc': 'Percentage Population 6-15',
    '15-18_perc': 'Percentage Population 15-18',
    '18-27_perc': 'Percentage Population 18-27',
    '27-45_perc': 'Percentage Population 27-45',
    '45-55_perc': 'Percentage Population 45-55',
    '55-65_perc': 'Percentage Population 55-65',
    '65_above_perc': 'Percentage Population > 65',
    'children_perc': 'Percentage Population < 18',
    'millenials_perc': 'Percentage Population Millenials',
    'boomers_perc': 'Percentage Population Baby Boomers',
    'gen_x_perc': 'Percentage Population Generation X', 
    
    'total': 'Total Population',
    'migr': 'with Migration Background',
    'no_migr': 'without Migration Background',
    'EU': 'EU',
    'Islamic_countries': 'Islamic Countries',
    'Other': 'Other',
    'former_JUGO': 'Former Jugoslavia',
    'former_UDSSR': 'Former Soviet Union',
    'AUT': 'Austria',
    'BGR': 'Bulgaria',
    'DEU': 'Germany',
    'ESP': 'Spain',
    'EU_other': 'Other EU',
    'FRA': 'France',
    'GRC': 'Greece',
    'HRV': 'Croatia',
    'ITA': 'Italy',
    'POL': 'Poland',
    'ROU': 'Romania',
    'UK': 'UK',
    'IRN': 'Iran',
    'Islamic_other': 'Other Islamic Countries',
    'LBN': 'Libanon', 
    'SYR': 'Syria',
    'TUR': 'Turkey',
    'USA': 'USA',
    'VNM': 'Vietnam', 
    'not_identified': 'unknown', 
    'other': 'Other', 
    'BIH': 'Bosnia-Herzegovina',
    'JUGO_other': 'Other Former Jugoslavia',
    'SRB': 'Serbia', 
    'KAZ': 'Kazakhstan', 
    'RUS': 'Russia', 
    'UDSSR_other': 'Other Former Soviet Union',
    'UKR': 'Ukraine'
}

In [40]:
features_short = {
    'Square Meter Price (in EUR)': 'sqm_price_all',
    'Total Population': 'total',
    'Population per km2': 'pop_per_sqm',
    'Area (in km2)': 'area_sqkm',
    'Square Meter Price (in EUR) - lower': 'sqm_price_lower',
    'Square Meter Price (in EUR) - upper': 'sqm_price_upper',
    'Rent cold (in EUR)': 'rent_cold',
    'Apartment Size (in m2)': 'apt_size',
    'Household Net Income (in EUR)': 'net_income',
    'Percentage Upper Quality Housing': 'upper_quality',
    'Percentage Pop. with Migration History': 'total_foreigners_perc',
    'Percentage Park Area': 'leisure_part',
    
    '<6': 'below_6',
    '65+': '65_above',
    
    'Age Bracket <6': 'below_6',
    'Age Bracket 6-15': '6-15',
    'Age Bracket 15-18': '15-18',
    'Age Bracket 18-27': '18-27',
    'Age Bracket 27-45': '27-45',
    'Age Bracket 45-55': '45-55',
    'Age Bracket 55-65': '55-65',
    'Age Bracket >65': '65_above',
    'Percentage Population Millenials': 'millenials',
    'Percentage Population Baby Boomers': 'boomers',
    'Percentage Population Generation X': 'gen_x',
    'Percentage Population < 6': 'below_6_perc',
    'Percentage Population 6-15': '6-15_perc',
    'Percentage Population 15-18': '15-18_perc',
    'Percentage Population 18-27': '18-27_perc',
    'Percentage Population 27-45': '27-45_perc',
    'Percentage Population 45-55': '45-55_perc',
    'Percentage Population 55-65': '55-65_perc',
    'Percentage Population > 65': '65_above_perc',
    'Percentage Population < 18': 'children_perc',
    
    'Children': 'children',
    'Millenials': 'millenials',
    'Baby<br>Boomers': 'boomers',
    'Generation X': 'gen_x',
    'Children (%)': 'children_perc',
    'Millenials (%)': 'millenials_perc',
    'Generation X (%)': 'gen_x_perc',
    'Baby<br>Boomers (%)': 'boomers_perc',
    
    '< 6 (%)': 'below_6_perc',
    '6-15 (%)': '6-15_perc',
    '15-18 (%)': '15-18_perc',
    '18-27 (%)': '18-27_perc',
    '27-45 (%)': '27-45_perc',
    '45-55 (%)': '45-55_perc',
    '55-65 (%)': '55-65_perc',
    '65+ (%)': '65_above_perc',
    
    'with Migration Background': 'migr',
    'without Migration Background': 'no_migr',
    'EU': 'EU',
    'Islamic Countries': 'Islamic_countries',
    'Other': 'other',
    'Former Jugoslavia': 'former_JUGO',
    'Former Soviet Union': 'former_UDSSR',
    'Austria': 'AUT',
    'Bulgaria': 'BGR',
    'Germany': 'DEU',
    'Spain': 'ESP',
    'Other EU': 'EU_other',
    'France': 'FRA',
    'Greece': 'GRC',
    'Croatia': 'HRV',
    'Italy': 'ITA',
    'Poland': 'POL',
    'Romania': 'ROU',
    'UK': 'UK',
    'Iran': 'IRN',
    'Other Islamic Countries': 'Islamic_other',
    'Libanon': 'LBN',
    'Syria': 'SYR',
    'Turkey': 'TUR',
    'USA': 'USA',
    'Vietnam': 'VNM',
    'unknown': 'not_identified',
    'Bosnia-Herzegovina': 'BIH',
    'Other Former Jugoslavia': 'JUGO_other',
    'Serbia': 'SRB',
    'Kazakhstan': 'KAZ',
    'Russia': 'RUS',
    'Other Former Soviet Union': 'UDSSR_other',
    'Ukraine': 'UKR',
    
    'Cluster': 'cluster',
    'GentriFactor': 'factor',
}

In [41]:
missing = {}
for key, value in features_long.items():
    if value not in features_short.keys():
        missing[value] = key

missing

{}

### Save dictionaries

In [42]:
import json

with open('./dataframes/lor_mapper.json', 'w') as f:
    json.dump(lor_mapper,f)
    
with open('./dataframes/region_mapper.json', 'w') as f:
    json.dump(region_mapper,f)
    
with open('./dataframes/migr_mapper.json', 'w') as f:
    json.dump(migr_mapper,f)
    
with open('./dataframes/age_mapper.json', 'w') as f:
    json.dump(age_mapper,f)
    
with open('./dataframes/features_short2long.json', 'w') as f:
    json.dump(features_long,f)
    
with open('./dataframes/features_long2short.json', 'w') as f:
    json.dump(features_short,f)

### Creating Data Dataframe

#### Percentage Change to Prior Year

In [43]:
df_data = df_perc.copy()

cols = [c for c in df_data.columns if c not in ['Bezirk', 'Bezirksregion', 'LOR', 'Bezirksname', 'year', 'urban_style', 'Name']]

for col in tqdm(cols):
    for l in df_perc['LOR'].unique():
        current_year = df_data.loc[df_perc['LOR']==l, col]
        prior_year = df_data.loc[df_perc['LOR']==l, col].shift(1)
        df_data.loc[df_data['LOR']==l, f'{col}_cng'] = (current_year - prior_year) / prior_year
    #replace inf values with the max of the column
    df_data[f'{col}_cng'].replace(np.inf, sorted(df_data[f'{col}_cng'].unique())[-2], inplace=True)

100%|██████████| 112/112 [01:06<00:00,  1.68it/s]


In [44]:
joblib.dump(df_data, './dataframes/df_data.csv')

['./dataframes/df_data.csv']

### Creating Clustering Dataframes

In [45]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [46]:
clustering_features = ['Bezirksname', 'Name', 'year',
                       '65_above_perc', 'lower_quality', 'sqm_price_all', 'net_income',
                       'USA_perc', 'DEU_perc', 'EU_perc', 'upper_quality', 
                       'total_foreigners_perc', '27-45_perc', 'pop_per_sqm']

df = df_perc.loc[(df_perc['year']>2009) & (df_perc['Name']!='Berlin'), clustering_features]
df.fillna(0, inplace=True)

info, features = df.iloc[:, :3], df.iloc[:, 3:]

# scaling the dataframe
df_scaled = StandardScaler().fit_transform(features)

joblib.dump(np.concatenate((info, df_scaled), axis=1), './dataframes/df_clustering.csv')

['./dataframes/df_clustering.csv']

In [47]:
pca = PCA(n_components=2, random_state=42)
principal_components = pca.fit_transform(df_scaled)

joblib.dump(principal_components, './dataframes/principal_components.csv')

['./dataframes/principal_components.csv']

### GentriFactor

In [48]:
df_gentrifactor = pd.DataFrame(data=np.concatenate((info, principal_components), axis=1),
                               columns=['Bezirksname', 'Name', 'year', 'pc_1', 'pc_2'])
df_gentrifactor[['pc_1', 'pc_2']] = df_gentrifactor[['pc_1', 'pc_2']].astype('float')

df_gentrifactor['factor'] = (df_gentrifactor['pc_1']+3).pow(3) + 1/np.cosh(df_gentrifactor['pc_2'])

#setting factor to max(factor, 0) for values < 0
df_gentrifactor.loc[df_gentrifactor['factor']<0,'factor'] = 0

gentri_factor = df_gentrifactor['factor'].to_numpy().reshape(-1,1)

In [49]:
joblib.dump(gentri_factor, './dataframes/gentri_factor.csv')

['./dataframes/gentri_factor.csv']